In [1]:
# set up
import os
import time
import numpy as np
import pandas as pd
import functools as func
import functions_v3 as f
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
home_path = "C:\\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\\07_Arbeitsordner\\04_Ergebnisse\\20220920_Ergebnisse_4_Wochen"
os.chdir(home_path)

#### KPI 4: CO2 emissions
---
Combined data frame for every entity in use case with:
- grey_energy [kWh] -> to be multiplied by co2 emission factor
- grey_energy [%] -> share grey energy consumed (1 - share renewable)

In [4]:
os.chdir(home_path)
os.chdir('use_case_6/')
use_case_home_path = os.getcwd()
df6 = f.co2_emissions(use_case_home_path)
df6

OSError: [Errno 22] Invalid argument

---

In [ ]:
# define function for execution convenience
def calculate_kpi_3_4(use_case_string):
    os.chdir(home_path)
    os.chdir(use_case_string)
    use_case_home_path = os.getcwd()
    print(f'-- use case {f.get_use_case_nr(use_case_home_path)} --')
    start = time.time()
    df = f.co2_emissions(use_case_home_path)
    print(f'time: {round(time.time() - start, 2)} seconds')
    return df

In [ ]:
# base case
df0 = calculate_kpi_3_4('Base_Case')
df0

In [ ]:
# use case 3
df3 = calculate_kpi_3_4('use_case_3')
df3

In [ ]:
# use case 5
df5 = calculate_kpi_3_4('use_case_5')
df5